In [1]:
#pip install geopandas
#pip install --upgrade pip

In [2]:
import pandas as pd
import numpy as np
#what statsmodels do I need for the below.
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from numpy.random import randint
from matplotlib.patheffects import withStroke
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [3]:
##heat map idea

#Summarize each by state and year (just doing 2019)
#combined states data by quality and net generation
#put that on a heatmap

In [4]:
d1 = pd.read_excel('cooling_data_2019.xlsx',header=2)

C:\Users\chrss\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [19]:
d1.columns
#d1

Index(['\n \n \n \n \n \nUtility ID', 'State', 'Plant Code', 'Plant Name',
       'Year', 'Month', 'Generator ID', 'Boiler ID', 'Cooling ID',
       'Generator Primary Technology',
       'Summer Capacity of Steam Turbines (MW)',
       'Gross Generation from Steam Turbines (MWh)',
       'Net Generation from Steam Turbines (MWh)',
       'Summer Capacity Associated with Single Shaft Combined Cycle Units (MW)',
       'Gross Generation Associated with Single Shaft Combined Cycle Units (MWh)',
       'Net Generation Associated with Single Shaft Combined Cycle Units (MWh)',
       'Summer Capacity Associated with Combined Cycle Gas Turbines (MW)',
       'Gross Generation Associated with Combined Cycle Gas Turbines (MWh)',
       'Net Generation Associated with Combined Cycle Gas Turbines (MWh)',
       'Fuel Consumption from All Fuel Types (MMBTU)',
       'Fuel Consumption from Steam Turbines (MMBTU)',
       'Fuel Consumption from Single Shaft Combined Cycle Units (MMBTU)',
       'Fu

In [6]:
d1['uniquekey1'] = [str(d1.loc[x][1]) + str(d1.loc[x][2]) + str(d1.loc[x][3]) + str(d1.loc[x][9])
                    for x in range(len(d1.index))]
#d1['uniquekey1']

In [17]:
#d2 = d1.fillna(0)
d1['Net Generation from Steam Turbines (MWh)'] = [x if int((str(x).strip()+'0')) > 0 else 0 
                                                  for x in d1['Net Generation from Steam Turbines (MWh)']]
d1['Net Generation Associated with Single Shaft Combined Cycle Units (MWh)'] = [x if int((str(x).strip()+'0')) > 0 else 0 
                                                  for x in d1['Net Generation Associated with Single Shaft Combined Cycle Units (MWh)']]
d1['Net Generation Associated with Combined Cycle Gas Turbines (MWh)'] = [x if int((str(x).strip()+'0')) > 0 else 0 
                                                  for x in d1['Net Generation Associated with Combined Cycle Gas Turbines (MWh)']]

d1['Total Generation'] = d1['Net Generation from Steam Turbines (MWh)'] + d1['Net Generation Associated with Single Shaft Combined Cycle Units (MWh)'] + d1['Net Generation Associated with Combined Cycle Gas Turbines (MWh)']
#pd.to_numeric(d2['Net Generation from Steam Turbines (MWh)'])
#d1['Net Generation from Steam Turbines (MWh)']

dgroup = d1.drop_duplicates(subset = 'uniquekey1')
x = dgroup.groupby(['State','Generator Primary Technology'])['Generator Primary Technology'].count().sort_values().groupby(level=0).tail(1)
x = pd.DataFrame(x)
x.rename(columns={'Generator Primary Technology': 'Generator Count'}, inplace=True)
x.reset_index(inplace=True)
#x

x2 =  d1.groupby(['State','Generator Primary Technology'])['Total Generation'].sum().sort_values().groupby(level=0).tail(1)
x2 = pd.DataFrame(x2)
x2.reset_index(inplace=True)



In [12]:
##map these with geopandas

#1 - create a map of the highest quantity generators at plants by state
x

,State,Generator Primary Technology,Generator Count
0,SD,Conventional Steam Coal,1
1,ME,Wood/Wood Waste Biomass,2
2,HI,Petroleum Liquids,2
3,ID,Natural Gas Fired Combined Cycle,2
4,DE,Natural Gas Fired Combined Cycle,2
5,MT,Conventional Steam Coal,2
6,NH,Natural Gas Fired Combined Cycle,2
7,RI,Natural Gas Fired Combined Cycle,3
8,TN,Conventional Steam Coal,4
9,NM,Natural Gas Steam Turbine,4


In [10]:
##map these with geopandas

#2 - create a map of the highest net generation volume by plant type by state
x2

,State,Generator Primary Technology,Net Generation from Steam Turbines (MWh)
0,ME,Wood/Wood Waste Biomass,1091716
1,DE,Natural Gas Fired Combined Cycle,1105476
2,ID,Natural Gas Fired Combined Cycle,1163046
3,RI,Natural Gas Fired Combined Cycle,1977951
4,SD,Conventional Steam Coal,2619088
5,MA,Natural Gas Fired Combined Cycle,2784855
6,HI,Petroleum Liquids,3325338
7,OR,Natural Gas Fired Combined Cycle,5758729
8,NV,Natural Gas Fired Combined Cycle,7674231
9,WA,Nuclear,8866499


In [ ]:
## Displaying Feature Investigation by Type using KNN

#calculate some key features such as Power Output / Capacity & Water Cooling data
#group on a KNN graph by type
#find the best looking KNN graph by type to find key features for...


In [21]:
#taking this list from the random forest by Brendan
keyfeatureslist = ['Generator Primary Technology',
       'Net Generation from Steam Turbines (MWh)',
       'Net Generation Associated with Single Shaft Combined Cycle Units (MWh)',
       'Net Generation Associated with Combined Cycle Gas Turbines (MWh)',
       'Summer Capacity of Steam Turbines (MW)',
       'Summer Capacity Associated with Single Shaft Combined Cycle Units (MW)',
       'Summer Capacity Associated with Combined Cycle Gas Turbines (MW)',
       'Fuel Consumption from All Fuel Types (MMBTU)',
       'Fuel Consumption from Steam Turbines (MMBTU)',
       'Fuel Consumption from Single Shaft Combined Cycle Units (MMBTU)',
       'Fuel Consumption from Combined Cycle Gas Turbines (MMBTU)',
       'Coal Consumption (MMBTU)', 'Natural Gas Consumption (MMBTU)',
       'Petroleum Consumption (MMBTU)', 'Biomass Consumption (MMBTU)',
       'Water Withdrawal Volume (Million Gallons)',
       'Water Consumption Volume (Million Gallons)',
       'Water Withdrawal Intensity Rate (Gallons / MWh)',
       'Water Consumption Intensity Rate (Gallons / MWh)',
       'Cooling Unit Hours in Service',
       'Average Distance of Water Intake Below Water Surface (Feet)']

#keyfeatureslist[1:]

In [30]:
#narrowing the apples/oranges list:
keygeneratorslist = ['Conventional Steam Coal', 'Natural Gas Fired Combined Cycle',
                     'Natural Gas Steam Turbine','Petroleum Liquids', 'Nuclear', 
                     'Wood/Wood Waste Biomass']
d2 = d1[d1['Generator Primary Technology'].isin(keygeneratorslist)]
d2 = d2[keyfeatureslist]

for column in keyfeatureslist:
    if column != keyfeatureslist[0]:
        d2[column] = [x if float((str(x).strip()+'0')) > 0 else 0 for x in d2[column]]
    else:
        continue
d2

,Generator Primary Technology,Net Generation from Steam Turbines (MWh),Net Generation Associated with Single Shaft Combined Cycle Units (MWh),Net Generation Associated with Combined Cycle Gas Turbines (MWh),Summer Capacity of Steam Turbines (MW),Summer Capacity Associated with Single Shaft Combined Cycle Units (MW),Summer Capacity Associated with Combined Cycle Gas Turbines (MW),Fuel Consumption from All Fuel Types (MMBTU),Fuel Consumption from Steam Turbines (MMBTU),Fuel Consumption from Single Shaft Combined Cycle Units (MMBTU),...,Coal Consumption (MMBTU),Natural Gas Consumption (MMBTU),Petroleum Consumption (MMBTU),Biomass Consumption (MMBTU),Water Withdrawal Volume (Million Gallons),Water Consumption Volume (Million Gallons),Water Withdrawal Intensity Rate (Gallons / MWh),Water Consumption Intensity Rate (Gallons / MWh),Cooling Unit Hours in Service,Average Distance of Water Intake Below Water Surface (Feet)
0,Natural Gas Steam Turbine,0,0,0,160.0,0,0,0,0,0,...,0,0,0,0,2715.910,0.000,0,0,739,10.0
1,Natural Gas Steam Turbine,0,0,0,160.0,0,0,0,0,0,...,0,0,0,0,1326.360,0.000,0,0,375,10.0
2,Natural Gas Steam Turbine,3916,0,0,160.0,0,0,70478,70478,0,...,0,70478,0,0,1536.050,0.000,392250,0,187,10.0
3,Natural Gas Steam Turbine,0,0,0,160.0,0,0,0,0,0,...,0,0,0,0,2469.600,0.000,0,0,720,10.0
4,Natural Gas Steam Turbine,12087,0,0,160.0,0,0,202310,202310,0,...,0,202310,0,0,3461.460,0.000,286379,0,325,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14698,Natural Gas Fired Combined Cycle,0,355383,0,0.0,485,0,2234604,0,2234604,...,0,2234604,0,0,73.031,60.755,0,0,744,0.0
14699,Natural Gas Fired Combined Cycle,0,328128,0,0.0,485,0,2072935,0,2072935,...,0,2072935,0,0,67.262,54.864,0,0,720,0.0
14700,Natural Gas Fired Combined Cycle,0,201793,0,0.0,485,0,1282754,0,1282754,...,0,1282754,0,0,45.310,35.132,0,0,495,0.0
14701,Natural Gas Fired Combined Cycle,0,312870,0,0.0,485,0,1995500,0,1995500,...,0,1995500,0,0,57.326,45.792,0,0,700,0.0


In [34]:
X = d2[keyfeatureslist[1:]]
y = d2["Generator Primary Technology"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
k = 5
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train,y_train)

KNeighborsClassifier()

In [35]:
knn.score(X_test,y_test)

0.9619154298515822

In [36]:
#checking other scores
k_range = range(1, 20)
scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    scores.append(knn.score(X_test, y_test))
    
scores

[0.9675161019322319,
 0.9599551946233548,
 0.9649957994959395,
 0.9624754970596472,
 0.9619154298515822,
 0.9596751610193223,
 0.9582749929991599,
 0.957714925791095,
 0.9557546905628675,
 0.9532343881265752,
 0.9504340520862503,
 0.9481937832539905,
 0.9445533464015682,
 0.9448333800056007,
 0.9448333800056007,
 0.9420330439652759,
 0.9400728087370485,
 0.939232707924951,
 0.9378325399047885]

In [ ]:
## Logistic Regression for anomaly detection

#What is an anomaly going to be for __ type of system

In [ ]:

# What are we looking at for unsupervised?